# Feature Transformation with Amazon a SageMaker Processing Job

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Scikit-Learn, Spark, Ray, and others are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of HuggingFace in a managed SageMaker environment to run our processing workload.

# NOTE:  THIS NOTEBOOK WILL TAKE A 5-10 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

![](img/processing.jpg)

## Contents

1. Setup Environment
1. Setup Input Data
1. Setup Output Data
1. Build a Scikit-Learn container for running the processing job
1. Run the Processing Job using Amazon SageMaker
1. Inspect the Processed Output Data

# Setup Environment

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [2]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region, 
                            config=config)
s3 = boto3.Session().client(service_name="s3", 
                            region_name=region,
                            config=config)

# Setup Input Data

In [3]:
from datasets import concatenate_datasets, load_dataset
dataset = load_dataset("knkarthick/dialogsum")
dataset = concatenate_datasets([dataset['train'], dataset['test'], dataset['validation']])
!mkdir data-summarization
dataset = dataset.train_test_split(0.5, seed=1234)
dataset['test'].to_csv('./data-summarization/dialogsum-1.csv', index=False)
dataset['train'].to_csv('./data-summarization/dialogsum-2.csv', index=False)
sess.upload_data('./data-summarization/dialogsum-1.csv', bucket=bucket, key_prefix='data-summarization')
sess.upload_data('./data-summarization/dialogsum-2.csv', bucket=bucket, key_prefix='data-summarization')

Using custom data configuration knkarthick--dialogsum-6d41e9a7b96e340e
Found cached dataset csv (/root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-6d41e9a7b96e340e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

mkdir: cannot create directory ‘data-summarization’: File exists


Loading cached split indices for dataset at /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-6d41e9a7b96e340e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-bb9bf4f1031fa64f.arrow and /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-6d41e9a7b96e340e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-43b99073b24e8c87.arrow


Creating CSV from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

's3://sagemaker-us-east-1-371366150581/data-summarization/dialogsum-2.csv'

In [4]:
raw_input_data_s3_uri = f's3://{bucket}/data-summarization/'
print(raw_input_data_s3_uri)

s3://sagemaker-us-east-1-371366150581/data-summarization/


In [5]:
!aws s3 ls $raw_input_data_s3_uri

2023-04-14 00:26:23    6544107 dialogsum-1.csv
2023-04-14 00:26:24    6572423 dialogsum-2.csv


# Load Other Variables

In [6]:
%store -r model_checkpoint

In [7]:
try:
    model_checkpoint
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [8]:
print(model_checkpoint)

google/flan-t5-base


In [9]:
# %store -r dataset_templates_name

In [10]:
# try:
#     dataset_templates_name
# except NameError:
#     print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
#     print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
#     print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [11]:
# print(dataset_templates_name)

In [12]:
# %store -r prompt_template_name

In [13]:
# try:
#     prompt_template_name
# except NameError:
#     print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
#     print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
#     print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [14]:
# print(prompt_template_name)

# Run the Processing Job using Amazon SageMaker

Next, use the Amazon SageMaker Python SDK to submit a processing job using our custom python script.

# Review the Processing Script

In [15]:
!pygmentize preprocess.py

import subprocess
import sys
import json
import argparse

subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers==4.26.1", "datasets==2.9.0", "torch==1.13.1"])

from transformers import AutoTokenizer
from datasets import load_dataset, DatasetDict
import os
import time


def transform_dataset(input_data,
                      output_data,
                      huggingface_model_name,
                      train_split_percentage,
                      test_split_percentage,
                      validation_split_percentage,
                      ):

    # load in the original dataset
    dataset = load_dataset(input_data)
    print(f'Dataset loaded from path: {input_data}\n{dataset}')
    
    # Load the tokenizer
    print(f'Loading the tokenizer for the model {huggingface_model_name}')
    tokenizer = AutoTokenizer.from_pretrained(huggingface_model_name)
    
    # make train test validation split
    train_testvalid = dataset['train'].train_test_split(1 - train_s

Run this script as a processing job.  You also need to specify one `ProcessingInput` with the `source` argument of the Amazon S3 bucket and `destination` is where the script reads this data from `/opt/ml/processing/input` (inside the Docker container.)  All local paths inside the processing container must begin with `/opt/ml/processing/`.

Also give the `run()` method a `ProcessingOutput`, where the `source` is the path the script writes output data to.  For outputs, the `destination` defaults to an S3 bucket that the Amazon SageMaker Python SDK creates for you, following the format `s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name>/`.  You also give the `ProcessingOutput` value for `output_name`, to make it easier to retrieve these output artifacts after the job is run.

The arguments parameter in the `run()` method are command-line arguments in our `preprocess.py` script.

Note that we sharding the data using `ShardedByS3Key` to spread the transformations across all worker nodes in the cluster.

In [16]:
processing_instance_type = "ml.c5.2xlarge"
processing_instance_count = 2
train_split_percentage = 0.9
validation_split_percentage = 0.05
test_split_percentage = 0.05

In [17]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    env={"AWS_DEFAULT_REGION": region},
    max_runtime_in_seconds=7200,
)

In [18]:
input_s3 = f's3://{bucket}/data-summarization/'

In [19]:
!aws s3 ls {input_s3}

2023-04-14 00:26:23    6544107 dialogsum-1.csv
2023-04-14 00:26:24    6572423 dialogsum-2.csv


In [20]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

processor.run(
    code="preprocess.py",
    inputs=[
        ProcessingInput(
            input_name="raw-input-data",
            source=raw_input_data_s3_uri,
            destination="/opt/ml/processing/input/data/",
            s3_data_distribution_type="ShardedByS3Key",
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train", 
            s3_upload_mode="EndOfJob", 
            source="/opt/ml/processing/output/data/train"
        ),
        ProcessingOutput(
            output_name="validation",
            s3_upload_mode="EndOfJob",
            source="/opt/ml/processing/output/data/validation",
        ),
        ProcessingOutput(
            output_name="test", 
            s3_upload_mode="EndOfJob", 
            source="/opt/ml/processing/output/data/test"
        ),
    ],
    arguments=[
        "--train-split-percentage",
        str(train_split_percentage),
        "--validation-split-percentage",
        str(validation_split_percentage),
        "--test-split-percentage",
        str(test_split_percentage),
        "--model-checkpoint",
        str(model_checkpoint),
        # "--balance-dataset",
        # str(balance_dataset),
        # "--dataset-templates-name",
        # str(dataset_templates_name),
        # "--prompt-template-name",
        # str(prompt_template_name),
    ],
    logs=True,
    wait=False,
)

INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2023-04-14-00-26-26-430


In [21]:
scikit_processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]
print(scikit_processing_job_name)

sagemaker-scikit-learn-2023-04-14-00-26-26-430


In [22]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, scikit_processing_job_name
        )
    )
)

In [23]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, scikit_processing_job_name
        )
    )
)

In [24]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, scikit_processing_job_name, region
        )
    )
)

# Monitor the Processing Job

In [25]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=scikit_processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

{'ProcessingInputs': [{'InputName': 'raw-input-data', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-371366150581/data-summarization/', 'LocalPath': '/opt/ml/processing/input/data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-371366150581/sagemaker-scikit-learn-2023-04-14-00-26-26-430/input/code/preprocess.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'train', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-371366150581/sagemaker-scikit-learn-2023-04-14-00-26-26-430/output/train', 'LocalPath': '/opt/ml/processing/output/data/train', 'S3UploadMode': 'EndOfJob'}, 'AppManaged': False}, {'OutputName': 'validation', 'S3O

In [ ]:
running_processor.wait(logs=False)

..........................................................................!

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# Inspect the Processed Output Data

Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [ ]:
processing_job_description = running_processor.describe()

output_config = processing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "train":
        processed_train_data_s3_uri = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "validation":
        processed_validation_data_s3_uri = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "test":
        processed_test_data_s3_uri = output["S3Output"]["S3Uri"]

print(processed_train_data_s3_uri)
print(processed_validation_data_s3_uri)
print(processed_test_data_s3_uri)

s3://sagemaker-us-east-1-371366150581/sagemaker-scikit-learn-2023-04-14-00-26-26-430/output/train
s3://sagemaker-us-east-1-371366150581/sagemaker-scikit-learn-2023-04-14-00-26-26-430/output/validation
s3://sagemaker-us-east-1-371366150581/sagemaker-scikit-learn-2023-04-14-00-26-26-430/output/test


In [ ]:
!aws s3 ls $processed_train_data_s3_uri/

2023-04-14 00:32:27    2545157 1681432344114.parquet
2023-04-14 00:32:37    2540571 1681432353780.parquet


In [ ]:
!aws s3 ls $processed_validation_data_s3_uri/

2023-04-14 00:32:27     150220 1681432344114.parquet
2023-04-14 00:32:37     150701 1681432353780.parquet


In [ ]:
!aws s3 ls $processed_test_data_s3_uri/

2023-04-14 00:32:27     153865 1681432344114.parquet
2023-04-14 00:32:38     157115 1681432353780.parquet


# Pass Variables to the Next Notebook(s)

In [ ]:
%store raw_input_data_s3_uri

Stored 'raw_input_data_s3_uri' (str)


In [ ]:
%store train_split_percentage

Stored 'train_split_percentage' (float)


In [ ]:
%store validation_split_percentage

Stored 'validation_split_percentage' (float)


In [ ]:
%store test_split_percentage

Stored 'test_split_percentage' (float)


In [ ]:
# %store balance_dataset

In [ ]:
%store processed_train_data_s3_uri

Stored 'processed_train_data_s3_uri' (str)


In [ ]:
%store processed_validation_data_s3_uri

Stored 'processed_validation_data_s3_uri' (str)


In [ ]:
%store processed_test_data_s3_uri

Stored 'processed_test_data_s3_uri' (str)


In [ ]:
%store

Stored variables and their in-db values:
ingest_create_athena_table_parquet_passed             -> True
model_checkpoint                                      -> 'google/flan-t5-base'
processed_test_data_s3_uri                            -> 's3://sagemaker-us-east-1-371366150581/sagemaker-s
processed_train_data_s3_uri                           -> 's3://sagemaker-us-east-1-371366150581/sagemaker-s
processed_validation_data_s3_uri                      -> 's3://sagemaker-us-east-1-371366150581/sagemaker-s
raw_input_data_s3_uri                                 -> 's3://sagemaker-us-east-1-371366150581/data-summar
role                                                  -> 'arn:aws:iam::371366150581:role/SageMakerRepoRole'
s3_private_path_tsv                                   -> 's3://sagemaker-us-east-1-371366150581/amazon-revi
s3_public_path_tsv                                    -> 's3://amazon-reviews-pds/tsv'
setup_dependencies_passed                             -> True
test_split_percentage